In [10]:
%pip install langchain_openai langchain_community langchain pymysql chromadb -q
%pip install -U langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install sqlite3
%pip install pandas
%pip install sqlalchemy

In [136]:
import os
from langchain_community.utilities.sql_database import SQLDatabase
# from langchain import LangSmithClient

os.environ["OPENAI_API_KEY"] = "sk-proj-LAr9BAArNj8UXWrl9DlKT3BlbkFJMhZrhwHoDm9JQLCBfoOu"

os.environ["LANGCHAIN_TRACING_V2"]= "true"

os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

os.environ["LANGCHAIN_API_KEY"]="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"

# client = LangSmithClient(api_key=os.getenv('LANGCHAIN_API_KEY'))

db_uri = r'sqlite:///C:\Users\digan\OneDrive\Documents\sqlite'

# sqlite:///C:/Users/digan/AppData/Roaming/DBeaverData/workspace6/.metadata/sample-database-sqlite-1/Chinook.db

# Initialize the SQLDatabase
db = SQLDatabase.from_uri(db_uri)

# print(db.dialect)
# print(db.get_usable_table_names())
# print(db.table_info)


In [140]:
# import openai
# from langchain.chains import create_sql_query_chain
# from langchain_openai import ChatOpenAI
# from langsmith.wrappers import wrap_openai
# from langsmith import traceable

# # Wrap the OpenAI client with LangSmith tracing
# client = wrap_openai(openai.Client())

# # Define your pipeline function with tracing
# @traceable  # Auto-trace this function
# def pipeline(user_input: str):
#     # Generate a response from the OpenAI model
#     response = client.chat.completions.create(
#         messages=[{"role": "user", "content": user_input}],
#         model="gpt-3.5-turbo"
#     )
#     return response.choices[0].message.content

# # Initialize the ChatOpenAI instance with the model
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# # Assume `db` is your database connection or any relevant object for the chain
# # You need to define `db` with your database connection details
# # Example:
# # from sqlalchemy import create_engine
# # db = create_engine('sqlite:///mydatabase.db')

# # Create the SQL query generation chain
# generate_query = create_sql_query_chain(llm, db)

# # Define the user input
# user_input = "what is price of `1968 Ford Mustang`"

# # Pass the user input through the pipeline to get a response
# pipeline_response = pipeline(user_input)

# # Invoke the SQL query generation chain
# query = generate_query.invoke({"question": user_input})

# # Print the generated query
# print(query)


from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "what is price of `1968 Ford Mustang`"})
# "what is price of `1968 Ford Mustang`"
print(query)


SELECT "productName", "buyPrice", "MSRP"
FROM products
WHERE "productName" = '1968 Ford Mustang'
LIMIT 1;


In [123]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

''

In [138]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "How many orders are there"})


'There are 0 orders in the database.'

In [126]:
examples = [
    {
        "input": "List all customers in France with a credit limit over 20,000.",
        "query": "SELECT * FROM customers WHERE country = 'France' AND creditLimit > 20000;"
    },
    {
        "input": "Get the highest payment amount made by any customer.",
        "query": "SELECT MAX(amount) FROM payments;"
    },
]


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input","top_k"],
    input_variables=["input"],
)
print(few_shot_prompt.format(input1="How many products are there?"))


In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="How many products are there?"))


In [ ]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info"))
generate_query = create_sql_query_chain(llm, db,final_prompt)
chain = (
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many customers with credit limit more than 50000"})


In [141]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("Genre_Info.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + f"Genre Name: " + row['Genre'] + "\n" + f"Year Created: " + row['YearCreated'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)


Genre Name: Rock
Year Created: 1950s

Genre Name: Jazz
Year Created: 1910s

Genre Name: Metal
Year Created: 1960s

Genre Name: Alternative & Punk
Year Created: 1970s

Genre Name: Rock And Roll
Year Created: 1950s

Genre Name: Blues
Year Created: 1920s

Genre Name: Latin
Year Created: 1940s

Genre Name: Reggae
Year Created: 1960s

Genre Name: Pop
Year Created: 1950s

Genre Name: Soundtrack
Year Created: 1920s

Genre Name: Bossa Nova
Year Created: 1950s

Genre Name: Easy Listening
Year Created: 1950s

Genre Name: Heavy Metal
Year Created: 1970s

Genre Name: R&B/Soul
Year Created: 1940s

Genre Name: Electronica/Dance
Year Created: 1980s

Genre Name: World
Year Created: 1980s

Genre Name: Hip Hop/Rap
Year Created: 1970s

Genre Name: Science Fiction
Year Created: 1920s

Genre Name: TV Shows
Year Created: 1940s

Genre Name: Sci Fi & Fantasy
Year Created: 1950s

Genre Name: Drama
Year Created: ~500 BCE

Genre Name: Comedy
Year Created: ~500 BCE

Genre Name: Alternative
Year Created: 1980s

Ge

In [142]:
table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

# {table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "give me the genres in the Genre table"})
tables

[Table(name='Genre')]

In [143]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "give me the name of the genre that was created in the 1500s"})

['Classical']

In [139]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many cutomers with order count more than 5"})


ValueError: table_names {'Orders', 'Customers'} not found in database